In [2]:
! pip install --quiet nltk

In [ ]:
import nltk 
nltk.download('wordnet')

In [9]:
import os
import pandas as pd
import numpy as np
from glob import glob
from nltk.stem import WordNetLemmatizer
import re

pd.set_option('max_colwidth',50)

In [12]:
class LemmaTokenizer(object):
    regex = r'\w+'
    def __init__(self):
        self.wnl = WordNetLemmatizer()        

    def __call__(self, doc):
        return [self.wnl.lemmatize(t) for t in re.findall(self.regex,doc)]
    
def get_indeed_texts(path, **kwargs):
    indeed = pd.read_csv(path, **kwargs)
    indeed['description'] = indeed.description.str.replace('Job Summary ', '')
    text = indeed.title.str.cat(indeed.description, sep=' ')
    return text, indeed

def get_exact_matches(df, lookup):
    merged = df.merge(lookup, how = 'left', on=['title'])
    return merged[~merged.code.isna()]

In [6]:
vectorizer = pd.read_pickle('models/tfidf.pkl')
policy_prob = pd.read_pickle('models/logistic-model.pkl')        

In [16]:
text,indeed = get_indeed_texts('data/2018-08-02T09-00-19.csv', nrows = 5000)
vecs = vectorizer.transform(text)
preds = policy_prob.predict(vecs)

In [22]:
d = pd.DataFrame({'predicted': preds, 'title': indeed.title, 'description': indeed.description})
d.head(50)

,predicted,title,description
0,Office and Administrative Support Occupations,Senior Accountant,We need experienced Accountant (M/F) for expor...
1,Office and Administrative Support Occupations,Finance Executive/ Sr. Accountant,"Accounts Manager- End to End Accounting, Produ..."
2,Office and Administrative Support Occupations,Senior Accountant,"Hi, Greetings From KINGFISH EXPORTS!! Urgently..."
3,Production Occupations,MIS Coordinator,Generate MIS documentation for smooth process ...
4,Business and Financial Operations Occupations,Senior Accountant,We are a chain of self serve frozen yogurt sto...
5,Office and Administrative Support Occupations,Senior Accountant,Senior Accountant Job Duties: Prepares and rec...
6,Office and Administrative Support Occupations,Accounts Assistant,Prepare and fact-check invoices before sending...
7,Office and Administrative Support Occupations,CA Articleship Trainee,Job Title CA Articleship Trainee \t \t Gender...
8,Computer and Mathematical Occupations,MIS Executive (Male Candidate),Vete Associates Tax & Investments Consultants ...
9,Business and Financial Operations Occupations,Accounts Assistant,"Dear Prospective Candidates, We are looking fo..."


In [ ]:
lookup = pd.read_csv('crosswalks/soc-title-lookup.csv')
v = get_exact_matches(indeed, lookup)
v